In [1]:
import pandas as pd

##### In order to set the name of the columns we use names parameter 
##### As in the file we have white arrow which actullay represent the tab that why we use sep = \t

In [2]:
df = pd.read_csv('smsspamcollection/SMSSpamCollection', names = ["label", "sms"], sep= "\t")
df.head()

,label,sms
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
df['sms'].loc[2]

"Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"

In [4]:
import nltk 
import re

In [5]:
from nltk.corpus import stopwords

In [6]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\seher\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

#### List of stop words of nltk 

In [7]:
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

## Now import stemmer to perform stemming

In [8]:
from nltk.stem.porter import PorterStemmer

In [9]:
ps = PorterStemmer()

### Appling stemming on the Data sets and using re Sub fn so accept only alphabets and discard or replace all other with space
-- Sub fn replace all other character with space except alphabets

-- lower all the character to avoid multiple words in vocabulary

-- as stemming performed on words that why we split it and convert into words or tokens 

-- Than we use string interpolation, first check if the word is not a stop word than pass it to perform stemming 

-- Againg combining all words into sentence in order to pass to our model

In [29]:
corpus = []
for i in range(0, len(df['sms'])):
    review = re.sub('[^a-zA-Z]', " ",df['sms'][i])
    
    ## converting it into lower case 
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word  in stopwords.words('english')]
    review = " ".join(review)
    corpus.append(review)

In [30]:
corpus[0]

'go jurong point crazi avail bugi n great world la e buffet cine got amor wat'

## Creating Bag of Words for our Data

In [31]:
from sklearn.feature_extraction.text import CountVectorizer

#### Count vectorizer sort the vocab according to the words occuring frequency so we are using the first 2500 words for our vocab

In [54]:
cv = CountVectorizer(max_features = 2500)
## As it's our independant data so using X

x = cv.fit_transform(corpus).toarray()

In [56]:
x[0]

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

#### AS our second column is Categorical Column so we are using ONE HOT Encoding

In [57]:
dummies = pd.get_dummies(df.label)
dummies

,ham,spam
0,1,0
1,1,0
2,0,1
3,1,0
4,1,0
...,...,...
5567,0,1
5568,1,0
5569,1,0
5570,1,0


In [58]:
dummies.drop('spam',axis='columns',inplace=True)

In [60]:
dummies.head()

,ham
0,1
1,1
2,0
3,1
4,1


In [61]:
y = dummies

In [69]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y , train_size = 0.7, random_state = 8)

In [70]:
x_train

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

## Creating MultinomialNB Model

In [71]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()

In [72]:
model.fit(x_train,y_train)

C:\Users\seher\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MultinomialNB()

In [73]:
model.predict(x_test)

array([1, 1, 1, ..., 1, 1, 1], dtype=uint8)

In [74]:
model.score(x_test,y_test)

0.9820574162679426

# Using TF-IFD Model Instead of Bag of Words

In [75]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [76]:
tfidf = TfidfVectorizer()

In [78]:
x = tfidf.fit_transform(corpus)

In [94]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y , train_size = 0.7, random_state = 0)

In [95]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB().fit(x_train, y_train)

C:\Users\seher\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [96]:
model.predict(x_test)

array([1, 1, 1, ..., 1, 1, 1], dtype=uint8)

In [97]:
model.score(x_test,y_test)

0.9700956937799043

In [101]:
y_predict = model.predict(x_test)

In [98]:
from sklearn.metrics import classification_report

### Classification Report

In [103]:
print(classification_report(y_test , y_predict))

              precision    recall  f1-score   support

           0       1.00      0.77      0.87       221
           1       0.97      1.00      0.98      1451

    accuracy                           0.97      1672
   macro avg       0.98      0.89      0.93      1672
weighted avg       0.97      0.97      0.97      1672



In [89]:
import gensim.downloader as api

wv = api.load('word2vec-google-news-300')

KeyboardInterrupt: 